### Week 6, Day 2

We're about to create and use our own MCP Server and MCP Client!

It's pretty simple, but it's not super-simple. The excitment around MCP is about how easy it is to share and use other MCP Servers - making our own does involve a bit of work.

Let's review some python code made mostly by a hard-working Engineering Team:

accounts.py

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

load_dotenv(override=True)

True

In [2]:
from accounts import Account # import a class from the script that you wrote

In [3]:
account = Account.get("Ed")
account

Account(name='ed', balance=9615.232, strategy='', holdings={'AMZN': 9}, transactions=[3 shares of AMZN at 18.036 each., 3 shares of AMZN at 72.144 each., 3 shares of AMZN at 38.076 each.], portfolio_value_time_series=[('2025-08-25 16:49:24', 10206.892), ('2025-08-25 16:49:27', 10083.892), ('2025-08-25 19:38:52', 10047.46), ('2025-08-25 19:38:52', 10083.46), ('2025-08-25 19:41:54', 10110.232), ('2025-08-25 19:41:55', 10470.232)])

In [4]:
account.buy_shares("AMZN", 3, "Because this bookstore website looks promising")

'Completed. Latest details:\n{"name": "ed", "balance": 9516.034, "strategy": "", "holdings": {"AMZN": 12}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 18.036, "timestamp": "2025-08-25 16:49:24", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 72.144, "timestamp": "2025-08-25 19:38:52", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 38.076, "timestamp": "2025-08-25 19:41:54", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 33.066, "timestamp": "2025-08-25 21:06:16", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-08-25 16:49:24", 10206.892], ["2025-08-25 16:49:27", 10083.892], ["2025-08-25 19:38:52", 10047.46], ["2025-08-25 19:38:52", 10083.46], ["2025-08-25 19:41:54", 10110.232], ["2025-08-25 19:41:55", 10470.232], ["2025-08-25 21:06:16", 1

In [5]:
account.report()

'{"name": "ed", "balance": 9516.034, "strategy": "", "holdings": {"AMZN": 12}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 18.036, "timestamp": "2025-08-25 16:49:24", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 72.144, "timestamp": "2025-08-25 19:38:52", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 38.076, "timestamp": "2025-08-25 19:41:54", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 33.066, "timestamp": "2025-08-25 21:06:16", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-08-25 16:49:24", 10206.892], ["2025-08-25 16:49:27", 10083.892], ["2025-08-25 19:38:52", 10047.46], ["2025-08-25 19:38:52", 10083.46], ["2025-08-25 19:41:54", 10110.232], ["2025-08-25 19:41:55", 10470.232], ["2025-08-25 21:06:16", 10608.034], ["2025-08-25 21:0

In [6]:
account.list_transactions()

[{'symbol': 'AMZN',
  'quantity': 3,
  'price': 18.036,
  'timestamp': '2025-08-25 16:49:24',
  'rationale': 'Because this bookstore website looks promising'},
 {'symbol': 'AMZN',
  'quantity': 3,
  'price': 72.144,
  'timestamp': '2025-08-25 19:38:52',
  'rationale': 'Because this bookstore website looks promising'},
 {'symbol': 'AMZN',
  'quantity': 3,
  'price': 38.076,
  'timestamp': '2025-08-25 19:41:54',
  'rationale': 'Because this bookstore website looks promising'},
 {'symbol': 'AMZN',
  'quantity': 3,
  'price': 33.066,
  'timestamp': '2025-08-25 21:06:16',
  'rationale': 'Because this bookstore website looks promising'}]

# convert funtions to mcp tools
you have `account` class in `account.py`.<br>
you create mcp tool based on this class in `account_server.py`:

```python
from mcp.server.fastmcp import FastMCP
from accounts import Account

mcp = FastMCP("accounts_server")

@mcp.tool()
async def get_balance(name: str) -> float:
    """Get the cash balance of the given account name.

    Args:
        name: The name of the account holder
    """
    return Account.get(name).balance

@mcp.tool()
async def get_holdings(name: str) -> dict[str, int]:
    """Get the holdings of the given account name.

    Args:
        name: The name of the account holder
    """
    return Account.get(name).holdings

@mcp.tool()
async def buy_shares(name: str, symbol: str, quantity: int, rationale: str) -> float:
    """Buy shares of a stock.

    Args:
        name: The name of the account holder
        symbol: The symbol of the stock
        quantity: The quantity of shares to buy
        rationale: The rationale for the purchase and fit with the account's strategy
    """
    return Account.get(name).buy_shares(symbol, quantity, rationale)


@mcp.tool()
async def sell_shares(name: str, symbol: str, quantity: int, rationale: str) -> float:
    """Sell shares of a stock.

    Args:
        name: The name of the account holder
        symbol: The symbol of the stock
        quantity: The quantity of shares to sell
        rationale: The rationale for the sale and fit with the account's strategy
    """
    return Account.get(name).sell_shares(symbol, quantity, rationale)

@mcp.tool()
async def change_strategy(name: str, strategy: str) -> str:
    """At your discretion, if you choose to, call this to change your investment strategy for the future.

    Args:
        name: The name of the account holder
        strategy: The new strategy for the account
    """
    return Account.get(name).change_strategy(strategy)

@mcp.resource("accounts://accounts_server/{name}")
async def read_account_resource(name: str) -> str:
    account = Account.get(name.lower())
    return account.report()

@mcp.resource("accounts://strategy/{name}")
async def read_strategy_resource(name: str) -> str:
    account = Account.get(name.lower())
    return account.get_strategy()

if __name__ == "__main__":
    mcp.run(transport='stdio')
```

### questions from code block above that define mcp tools and resources

Q. 🧩 What does `mcp.run` do?
```python
if __name__ == "__main__":
    mcp.run(transport="stdio")
```

#### 🔹 Starts the FastMCP server
`FastMCP` is a helper for building an **MCP (Model Context Protocol) server**.  
When you call `.run()`, it launches the server process and begins listening for requests from an MCP client (such as an AI assistant or another system that speaks MCP).

---

#### 🔹 Chooses the transport
The argument `transport='stdio'` means the server will communicate over **standard input and output streams (stdin/stdout)**.

- This is the most common way MCP servers are run, since it lets the server integrate easily with clients that just need to spawn the process and connect to it.  
- Other transports (like sockets or custom ones) might be possible, but here it’s using **stdio**.

---

#### 🔹 Registers tools and resources
Before calling `.run()`, you defined several tools (`get_balance`, `buy_shares`, etc.) and resources (`accounts://...`).  
When the server runs, all of those become **callable methods** that a client can invoke via the MCP protocol.



### Now we write an MCP server and use it directly!

#### Q. why we need `async with`? <br>
A. The key reason you cannot use a normal `with` here is because `MCPServerStdio` is an asynchronous context manager. <br>
`MCPServerStdio` is openAI agents SDK **asynchronous** context manager 

normal `with` returns type error: <br>
```bash
TypeError: 'MCPServerStdio' object does not support the context manager protocol
```

In [7]:
# # it return error: TypeError: 'MCPServerStdio' object does not support the context manager protocol
# params = {"command": "uv", "args": ["run", "accounts_server.py"]}
# with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
#     mcp_tools = server.list_tools()


In [8]:
# Now let's use our accounts server as an MCP server to get the list of tools

params = {"command": "uv", "args": ["run", "accounts_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()


In [9]:
for mcp_tool in mcp_tools:
    print('---')
    print(mcp_tool)


---
name='get_balance' description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ' inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'} annotations=None
---
name='get_holdings' description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ' inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'} annotations=None
---
name='buy_shares' description="Buy shares of a stock.\n\n    Args:\n        name: The name of the account holder\n        symbol: The symbol of the stock\n        quantity: The quantity of shares to buy\n        rationale: The rationale for the purchase and fit with the account's strategy\n    " inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}, 

In [10]:
instructions = "You are able to manage an account for a client, and answer questions about the account."
request = "My name is Ed and my account is under the name Ed. What's my balance and my holdings?"
model = "gpt-4.1-mini"

In [11]:

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server: # create mcp_server
    agent = Agent(name="account_manager", instructions=instructions, model=model, mcp_servers=[mcp_server]) # pass mcp_server to the agent
    with trace("account_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))


Ed, your current account balance is $9,516.03. In terms of holdings, you have 12 shares of Amazon (AMZN). If you need any further details or want to make transactions, just let me know!

### build our own MCP Client

In [ ]:
from accounts_client import get_accounts_tools_openai, read_accounts_resource, list_accounts_tools

mcp_tools = await list_accounts_tools()
print(mcp_tools)
openai_tools = await get_accounts_tools_openai()
print(openai_tools)

In [ ]:
request = "My name is Ed and my account is under the name Ed. What's my balance?"

with trace("account_mcp_client"):
    agent = Agent(name="account_manager", instructions=instructions, model=model, tools=openai_tools)
    result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

In [ ]:
context = await read_accounts_resource("ed")
print(context)

In [ ]:
from accounts import Account
Account.get("ed").report()

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercises</h2>
            <span style="color:#ff7800;">Make your own MCP Server! Make a simple function to return the current Date, and expose it as a tool so that an Agent can tell you today's date.<br/>Harder optional exercise: then make an MCP Client, and use a native OpenAI call (without the Agents SDK) to use your tool via your client.
            </span>
        </td>
    </tr>
</table>